In [ ]:
import sys
import os
import json

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..")))
from Raydium.adapter.RaydiumAdapter import RaydiumAdapter
from SaucerSwap.adapter.SaucerSwapAdapter import SaucerSwapAdapter

def print_pretty_position(title, position):
    """
    Imprime de forma legible y clara la información de una posición.
    Si la posición existe, muestra los detalles relevantes.
    Si no existe, muestra el motivo o error.
    """
    print("=" * 60)
    print(f"{title}")
    print("=" * 60)
    if position.get("exists"):
        print("✅ Posición encontrada:")
        # Imprime los detalles de forma legible (indentado)
        print(json.dumps(position["details"], indent=2, ensure_ascii=False))
    else:
        print("❌ Posición NO encontrada.")
        detalles = position.get("details", {})
        if detalles:
            print("Detalles:")
            print(json.dumps(detalles, indent=2, ensure_ascii=False))
    print("\n")

# Consulta y muestra la posición en Raydium
adapter = RaydiumAdapter("../Raydium/config/solana.raydium.yaml")
res_raydium = adapter.check_position_exists_tool(
    position_nft_mint="GBYTTW6jMYGZUN5p4sLCVR14LMtaJqLWpjzD79k7FyLw"
)
print_pretty_position("Raydium (Solana)", res_raydium)

# Consulta y muestra la posición en SaucerSwap
adapter = SaucerSwapAdapter("/root/Repositorios/LiquidityProvider/SaucerSwap/config/hedera.saucerswap.yaml")
res_saucer = adapter.check_position_exists_tool(serial=67538)
print_pretty_position("SaucerSwap (Hedera)", res_saucer)
